<a href="https://colab.research.google.com/github/Lorenc1o/DataMining_HackMyRide/blob/main/sanitize_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import auth
auth.authenticate_user() # Disable in vm

!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

!mkdir main
!gcsfuse --implicit-dirs datamining-ulb main

!ls /content/main

!pip install jenkspy
!pip install ckwrap
!pip install dbscan1d

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2426  100  2426    0     0  93307      0 --:--:-- --:--:-- --:--:-- 97040
OK
21 packages can be upgraded. Run 'apt list --upgradable' to see them.
gcsfuse is already the newest version (0.41.9).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
mkdir: cannot create directory ‘main’: File exists
2022/12/13 10:59:45.686713 Start gcsfuse/0.41.9 (Go version go1.18.4) for app "" using mount point: /content/main
2022/12/13 10:59:45.734930 Opening GCS connection...
2022/12/13 10:59:47.307817 Mounting file system "datamining-ulb"...
2022/12/13 10:59:47.310419 File system has been successfully mounted.
data	    Lines_vehiclePositions  ordered_lines  vehiclePositions
esri_files  normalized

In [3]:
import pandas as pd
import re
import numpy as np
import math
import datetime
import decimal

In [4]:
'''
 Helper functions & constants
'''

decimal.getcontext().prec = 3
TO_MS = decimal.Decimal("1000")
STOP_DISTANCE_THRESHOLD = 5 # In metres
AVERAGE_SPEED = decimal.Decimal("4.667") # in metre/second

def epoch_ms_to_datetime(epoch_ms: int) -> datetime.datetime:
    return datetime.datetime.fromtimestamp(epoch_ms / int(TO_MS))

In [ ]:
# Extract actual stops
actual_df = pd.read_csv("/content/main/data/actual_stops.csv")
size = actual_df.shape[0]
actual_df["stop_id"] = [re.sub("\D", "", x.strip()) for x in actual_df["stop_id"]]

actual_df["succession"] = actual_df["succession"].astype(str)
actual_df["numero_lig"] = actual_df["numero_lig"].astype(str)
actual_df["variante"] = actual_df["variante"].astype(str)
actual_stops = set(actual_df["stop_id"])

# Extract Line stop sequence
extracted_lines = {}

delimiters = np.array(["_" for _ in range(size)])
stops_info = (actual_df["numero_lig"].to_numpy() + delimiters +  actual_df["succession"].to_numpy() + delimiters + actual_df["variante"].to_numpy() + delimiters + actual_df["stop_id"].to_numpy()).tolist()

prev_succession = -999999
direction = ""
for stops in stops_info[::-1]:
    info = stops.split("_")
    if int(prev_succession) < int(info[1]):
        direction = info[3]
    
    data = extracted_lines.setdefault(info[0], {})
    
    succession_info = data.setdefault(info[3], {})
    succession_info[direction] = info[1]
    
    data[info[3]] = succession_info
    extracted_lines[info[0]] = data
    
    line_direction = data.setdefault("direction", {})
    visited_stops = line_direction.setdefault(direction, set())
    visited_stops.add(info[3])
    
    line_direction[direction] = visited_stops
    data["direction"] = line_direction
    
    prev_succession = info[1]

'''
Pair stops with the succession line, 
 1. if there's multiple sucession line check whether direction is the same
 2. If not, try to find direction id in the direction stops
 3. otherwise set sucession to 99999
'''

lines = []
names = set()
for file in glob("/content/main/ordered_lines/*.csv"):
  name = file.strip().split("/")[4].split("_")[0]
  names.add(name)

print(names)
for file in glob("/content/main/Lines_vehiclePositions/*.csv"):
    if "null" in file or file in names:
      continue

    df = pd.read_csv(file)
    df["lineID"] = df["lineID"].astype(str)
    df["directionID"] = df["directionID"].astype(str)
    df["pointID"] = df["pointID"].astype(str).apply(lambda x: x.strip().split(" ")[0])
    
    delimiters = np.array(["_" for _ in range(df.shape[0])])
    stops_infos = (df["lineID"].to_numpy() + delimiters + df["directionID"].to_numpy() + delimiters + df["pointID"].to_numpy() + delimiters + df.index.astype(str).values).tolist()
    
    succession_arr = []
    for stop in stops_infos:
        info = stop.split("_")
        data = extracted_lines[info[0]]
        
        succession = 999999
        if info[2] not in data:
            succession_arr.append(succession)
            continue
        
        succession_info = data[info[2]]
        if info[1] not in succession_info:
            succession_arr.append(succession)
            continue
        
        succession = succession_info[info[1]]
        succession_arr.append(succession)
    
    df["succession"] = np.array(succession_arr)
    
    df["lineID"] = df["lineID"].astype(int)
    df["directionID"] = df["directionID"].astype(int)
    df["pointID"] = df["pointID"].astype(int)
    df["succession"] = df["succession"].astype(int)

    name = file.strip().split("/")[4].split("_")[0]
    df.sort_values(["directionID", "time",  "succession",], ascending=[True, True, True]).to_csv(f"/content/main/ordered_lines/{name}_ordered.csv")
    

{'Line71', 'Line93', 'Line20', 'Line78', 'Line6', 'Line28', 'Line95', 'Line33', 'Line27', 'Line38', 'Line47', 'Line79', 'Line13', 'Line62', 'Line48', 'Line43', 'Line58', 'Line92', 'Line46', 'Line12', 'Line42', 'Line57', 'Line74', 'Line1', 'Line88', 'Line89', 'Line77', 'Line54', 'Line19', 'Line39', 'Line17', 'Line56', 'Line69', 'Line72', 'Line81', 'Line61', 'Line8', 'Line80', 'Line36', 'Line50', 'Line76', 'Line5', 'Line7', 'Line70', 'Line9', 'Line21', 'Line60', 'Line63', 'Line64', 'Line75', 'Line14', 'Line98', 'Line41', 'Line45', 'Line66', 'Line53', 'Line59', 'Line51', 'Line55', 'Line82', 'Line44', 'Line37', 'Line2', 'Line29', 'Line34', 'Line49', 'Line86', 'Line25', 'Line65', 'Line4', 'Line97', 'Line3', 'Line87', 'Line83'}


In [ ]:
# Setting 999999 to the right successsion where possible, same bus stop but pointId for different line

actual_df = pd.read_csv("/content/main/data/actual_stops.csv")
size = actual_df.shape[0]
actual_df["stop_id"] = [re.sub("\D", "", x.strip()) for x in actual_df["stop_id"]]
actual_df["succession"] = actual_df["succession"].astype(str)
actual_df["numero_lig"] = actual_df["numero_lig"].astype(str)
actual_df["variante"] = actual_df["variante"].astype(str)
actual_stops = set(actual_df["stop_id"])

for filename in glob("/content/main/ordered_lines/*.csv"):
  # filename = '/content/main/ordered_lines/Line71_ordered.csv'
  cur_lineID = str(int(filename.split('/')[-1].split('_')[0][4:]))
  df = pd.read_csv(filename)
  df["lineID"] = df["lineID"].astype(str)
  df["directionID"] = df["directionID"].astype(str)
  df["pointID"] = df["pointID"].astype(str).apply(lambda x: x.strip().split(" ")[0])
  print('Line ', cur_lineID)
  n_faulty = df.loc[df['succession']==999999].shape[0]
  n_all = df.shape[0]
  print(f'before {n_faulty} ({round((n_faulty/n_all)*100, 2)}%)')

  for idx, faulty_row in df.loc[df['succession']==999999].iterrows():
    try:
      lineID = faulty_row.loc['lineID']
      pointID = faulty_row.loc['pointID']
      pointID_desc = actual_df.loc[actual_df['stop_id'] == pointID].iloc[0]['descr_fr']
      directionID = faulty_row.loc['directionID']
      directionID_desc = actual_df.loc[actual_df['stop_id']==directionID].iloc[0]['descr_fr']
    except:
      continue

    # might be the same bus stop but for a different line
    # that's why, if exists, get another pointID that has the same description, lineID and terminus
    matched_stops = actual_df.loc[(actual_df['descr_fr']==pointID_desc) & (actual_df['numero_lig'] == lineID) & (actual_df['terminus'] == directionID_desc)]

    if not matched_stops.empty:
      # if such a bus stop is found
      if matched_stops.shape[0] != 1:
        # print('Too many matches!')
        continue
      new_succession = int(matched_stops.iloc[0]['succession'])
      df.at[idx, 'succession'] = new_succession
      continue
    
    # cur_directionId might be stop before the actual terminal
    # in that case check that there is another pointID that has the same description, lineID and pointID as cur_directionID
    # if pointID is the same, then it's the going with the same route
    cur_dirID_info = actual_df.loc[(actual_df['descr_fr']==directionID_desc) & (actual_df['numero_lig'] == lineID) & (actual_df['stop_id'] == directionID)]
    if not cur_dirID_info.empty:
      if cur_dirID_info.shape[0] != 1:
        # print('Too many matches for case 2')
        continue

      # the actual terminal stop
      main_dirID_desc = cur_dirID_info.iloc[0]['terminus']

      # find the target bus stop's succession with the actual terminal stop
      correct_stop_info = actual_df.loc[(actual_df['descr_fr']==pointID_desc) & (actual_df['terminus'] == main_dirID_desc) & (actual_df['numero_lig']==lineID)]
      if correct_stop_info.shape[0] != 1:
        # print('Too many or not enough matches for case 2 correct info')
        continue
      
      new_succession = int(correct_stop_info.iloc[0]['succession'])

      # and confirm that cur_directionID is greater in succession than the current pointID
      if int(cur_dirID_info.iloc[0]['succession']) < new_succession:
        # print('Terminal Stop Succession is smaller than earlier stop')
        continue
      
      df.at[idx, 'succession'] = new_succession
      continue
  n_faulty = df.loc[df['succession']==999999].shape[0]
  n_all = df.shape[0]
  print(f'after {n_faulty} ({round((n_faulty/n_all)*100, 2)}%)')
  df.sort_values(["directionID", "time",  "succession",], ascending=[True, True, True]).to_csv(f"/content/main/ordered_lines/Line{cur_lineID}_ordered.csv")

Line  12
before 5496 (2.3%)
after 5496 (2.3%)
Line  13
before 12387 (5.35%)
after 12387 (5.35%)
Line  14
before 38854 (14.26%)
after 26554 (9.74%)
Line  17
before 968 (1.03%)
after 324 (0.34%)
Line  19
before 221734 (59.27%)


KeyboardInterrupt: ignored

In [ ]:
for filename in glob("/content/main/ordered_lines/*.csv"):
  # filename = '/content/main/ordered_lines/Line71_ordered.csv'
  cur_lineID = str(int(filename.split('/')[-1].split('_')[0][4:]))
  df = pd.read_csv(filename)
  df["lineID"] = df["lineID"].astype(str)
  df["directionID"] = df["directionID"].astype(str)
  df["pointID"] = df["pointID"].astype(str).apply(lambda x: x.strip().split(" ")[0])
  print('Line ', cur_lineID)
  n_faulty = df.loc[df['succession']==999999].shape[0]
  n_all = df.shape[0]
  print(f'before {n_faulty} ({round((n_faulty/n_all)*100, 2)}%)')

Line  12
before 5496 (2.3%)
Line  13
before 12387 (5.35%)
Line  14
before 26554 (9.74%)
Line  17
before 324 (0.34%)
Line  19
before 221734 (59.27%)
Line  1
before 177016 (57.95%)
Line  20
before 26897 (11.19%)
Line  21
before 13809 (7.04%)
Line  25
before 387235 (100.0%)
Line  27
before 44504 (20.03%)
Line  28
before 13510 (7.16%)
Line  29
before 51606 (14.64%)
Line  2
before 25443 (11.83%)
Line  33
before 704 (1.01%)
Line  34
before 93732 (34.52%)
Line  36
before 73993 (30.42%)
Line  37
before 37662 (22.98%)
Line  38
before 86436 (24.93%)
Line  39
before 140251 (66.38%)
Line  3
before 258804 (67.48%)
Line  41
before 34311 (14.65%)
Line  42
before 10022 (8.02%)
Line  43
before 40637 (19.76%)
Line  44
before 88253 (47.22%)
Line  45
before 41233 (18.39%)
Line  46
before 72315 (19.33%)
Line  47
before 58683 (29.06%)
Line  48
before 84633 (24.62%)
Line  49
before 21177 (6.49%)
Line  4
before 231078 (81.67%)
Line  50
before 4577 (2.55%)
Line  51
before 110492 (22.09%)
Line  53
before 1473 (

In [ ]:
df

,Unnamed: 0,time,lineID,directionID,distancefromPoint,pointID,succession
0,34064,1631018950001,19,471,0,5103,999999
1,34078,1631018982872,19,471,0,5103,999999
2,34093,1631019015919,19,471,0,5103,999999
3,34107,1631019048512,19,471,0,5103,999999
4,34121,1631019079131,19,471,0,5103,999999
...,...,...,...,...,...,...,...
374095,353570,1632179202844,19,5700,0,5700,999999
374096,353571,1632179234281,19,5700,0,5700,999999
374097,353572,1632179266232,19,5700,0,5700,999999
374098,353574,1632179297260,19,5700,0,5700,999999


In [ ]:
faulty_row

Unnamed: 0                  131027
Unnamed: 0.1                 92676
time                 1631415236099
lineID                          12
directionID                   6465
distancefromPoint               84
pointID                       1317
succession                  999999
Name: 131027, dtype: object

In [ ]:
actual_df.loc[actual_df['stop_id'] == pointID]

,gid,code_ligne,variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,geom


In [5]:
'''
Normalize the distance
 The algorithm will work as follow:
 1. We will change the distance for non-zero distance point by subtracting the time with how much time passed since the bus in the 0 distance of the point
 2. We will use the bus/tram average speed to calculate the distance passed
'''
line_df = pd.read_csv("/content/main/ordered_lines/Line71_ordered.csv")

# Remove outlier
line_df = line_df[line_df["succession"] != 999999]

columns = ["time", "lineID", "directionID", "distancefromPoint", "pointID", "succession"]

size = line_df.shape[0]
delimiters = np.array(["_" for _ in range(size)])

line_infos = None
for idx, column in enumerate(columns):
    line_df[column] = line_df[column].astype(str)
    if line_infos is None:
        line_infos = line_df[column].to_numpy() + delimiters
    elif line_infos is not None and idx < len(columns)-1:
        line_infos += line_df[column].to_numpy() + delimiters
    else:
        line_infos += line_df[column].to_numpy()

        
# Array for building new dataset
converted_timestamp = []
converted_distances = []
direction_ids = []
point_ids = []
line_ids = []
orders = []
original_timestamp = []
original_distances = []

for line_info in line_infos:
    line = line_info.split("_")

    # Unpack information
    timestamp = int(line[0])
    line_id = line[1]
    direction_id = line[2]
    distance = decimal.Decimal(line[3])
    point_id = line[4]
    order = int(line[5])
    
    # Ignore stopping bus
    if int(distance) == 0:
        line_ids.append(line_id)
        direction_ids.append(direction_id)
        point_ids.append(point_id)
        orders.append(order)
        
        original_timestamp.append(epoch_ms_to_datetime(timestamp))
        converted_timestamp.append(epoch_ms_to_datetime(timestamp))
        original_distances.append(distance)  
        converted_distances.append(distance)
        continue
    
    # Calculate -5 meters and +5 meters from stop
    # The interpolation threshold is chosen by using heuristic method
    # We will consider bus position -5 metres or +5 metres from stop is considered as stopping    
    # Normalized the timestamp
    timestamp_plus = timestamp - (int(((distance+STOP_DISTANCE_THRESHOLD) / AVERAGE_SPEED)*TO_MS))
    timestamp_min = timestamp - (int(((distance-STOP_DISTANCE_THRESHOLD) / AVERAGE_SPEED)*TO_MS))
    
    timestamps = [timestamp_min, timestamp_plus]
    time_passed = [(int((distance-STOP_DISTANCE_THRESHOLD) / AVERAGE_SPEED)), (int((distance+STOP_DISTANCE_THRESHOLD) / AVERAGE_SPEED))] 
    
    # Add the interpolation data
    for idx in range(len(timestamps)):
        line_ids.append(line_id)
        direction_ids.append(direction_id)
        point_ids.append(point_id)
        orders.append(order)
        
        original_timestamp.append(epoch_ms_to_datetime(timestamp))
        converted_timestamp.append(epoch_ms_to_datetime(timestamps[idx]))
        
        original_distances.append(distance)   
        converted_distances.append(distance - (AVERAGE_SPEED*time_passed[idx]))

columns = {
            "line_id": np.array(line_ids), 
           "direction_id": np.array(direction_ids),
           "point_id": np.array(point_ids), 
           "order": np.array(orders), 
           "original_timestamp": np.array(original_timestamp), 
           "converted_timestamp": np.array(converted_timestamp), 
           "original_distance": np.array(original_distances), 
           "converted_distance": np.array(converted_distances)
          }


converted_df = pd.DataFrame(columns)
converted_df.sort_values(["direction_id", "order", "converted_timestamp"], ascending=[True, True, True]).to_csv("/content/main/normalized_lines/Line71_normalized.csv")

In [9]:
from numpy import array, linspace, sort
from sklearn.neighbors import KernelDensity
from matplotlib.pyplot import plot
from scipy.signal import argrelextrema
from jenkspy import JenksNaturalBreaks
from dbscan1d.core import DBSCAN1D

import ckwrap


normalized_df = pd.read_csv("/content/main/normalized_lines/Line71_normalized.csv")
normalized_df = normalized_df[(normalized_df["order"].astype(int) == 9)  & (normalized_df["direction_id"] == 2596 )]
normalized_df["converted_timestamp"] = normalized_df["converted_timestamp"].apply(lambda x: (pd.Timestamp(x)))

arrival_time = normalized_df['converted_timestamp'].apply(lambda x: (x - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')).to_numpy()

number_of_trips = pd.read_csv("/content/main/schedule/schedules/schedule_71_2596.csv")
n = len(number_of_trips["3562B"])


'''
    Jenkins binning algorithm
'''
jnb = JenksNaturalBreaks(n)
jnb.fit(arrival_time)


'''
 Kernel Density estimation method
'''
first = arrival_time[0]
a = np.array([(x-first) for x in arrival_time])

# a = np.array([1,1,1,2,3,4,10,12,16,40,45])
a = a.reshape(-1, 1)

kde = KernelDensity(kernel='gaussian', bandwidth=0.01).fit(a)

s = linspace(a[0],a[-1]+2000)
e = kde.score_samples(s.reshape(-1,1))
mi, ma = argrelextrema(e, np.less)[0], argrelextrema(e, np.greater)[0]

clusters = []

clusters.append(a[a < s[mi][0]])  # most left cluster

# print all middle cluster
for i_cluster in range(len(mi)-1):
    clusters.append(a[(a >= s[mi][i_cluster]) * (a <= s[mi][i_cluster+1])])

clusters.append(a[a >= s[mi][-1]])  # print most right cluster

'''
    dbsscan approach
    epsilon is 35 seconds 
'''
dbs = DBSCAN1D(eps=35, min_samples=1)
labels = dbs.fit_predict(arrival_time)

'''
    kmeans 1d
'''
first = arrival_time[0]
data = np.array([(x-first) for x in arrival_time])
result = ckwrap.ckmeans(data, n)

In [11]:
normalized_df["dbscan"] = np.array(dbs.labels_)
normalized_df["kmeans"] = np.array(result.labels)
normalized_df["jenkins"] = np.array(jnb.labels_)

kde_labels = []
for idx, cluster in enumerate(clusters):
    for x in cluster:
        kde_labels.append(idx)
        
normalized_df["KDE"] = np.array(kde_labels)

normalized_df.to_csv("/content/main/arrival_time/sample_cluster.csv")